#  BRITISH SIGN LANGUAGE DETECTION:


# Importing Libraries


In [60]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# MediaPipe Holistic Pose Estimation: Initializing Modules


In [61]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [62]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [63]:
def draw_landmarks(image, results):

    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [64]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [65]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [66]:
len(results.left_hand_landmarks.landmark)

21

In [67]:
draw_landmarks(frame, results)

In [68]:
# Check if left hand landmarks are detected before accessing its length
if results.left_hand_landmarks is not None:
    left_landmarks_count = len(results.left_hand_landmarks.landmark)
    print(f"Number of left hand landmarks detected: {left_landmarks_count}")
else:
    print("No left hand landmarks detected")

# Check if right hand landmarks are detected before accessing its length
if results.right_hand_landmarks is not None:
    right_landmarks_count = len(results.right_hand_landmarks.landmark)
    print(f"Number of right hand landmarks detected: {right_landmarks_count}")
else:
    print("No right hand landmarks detected")


Number of left hand landmarks detected: 21
No right hand landmarks detected


In [69]:
pose = []
for res in results.left_hand_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [70]:
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

# Extracting Keypoints

In [71]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [72]:
result_test = extract_keypoints(results)

In [73]:
result_test

array([ 6.85742557e-01,  6.61644220e-01,  5.17777437e-07,  6.13713086e-01,
        6.49409413e-01, -3.00781224e-02,  5.42656720e-01,  5.96693754e-01,
       -4.60462347e-02,  4.92046118e-01,  5.48380196e-01, -5.99723868e-02,
        4.45926428e-01,  5.26519895e-01, -7.40302578e-02,  5.73541284e-01,
        4.40660000e-01, -2.50169374e-02,  5.43529809e-01,  3.44234884e-01,
       -4.56706658e-02,  5.27406573e-01,  2.83522189e-01, -6.31972179e-02,
        5.15828252e-01,  2.30630711e-01, -7.68473372e-02,  6.18741572e-01,
        4.17345881e-01, -2.91871261e-02,  5.99099398e-01,  3.03525925e-01,
       -4.52282131e-02,  5.88448584e-01,  2.31210619e-01, -6.05011061e-02,
        5.80091417e-01,  1.73088878e-01, -7.27620646e-02,  6.63854718e-01,
        4.18142319e-01, -3.81020680e-02,  6.57098949e-01,  3.13628912e-01,
       -5.64806126e-02,  6.52132511e-01,  2.48621956e-01, -6.89935386e-02,
        6.45090997e-01,  1.93083942e-01, -7.86941946e-02,  7.08427429e-01,
        4.36829925e-01, -

# Importing libraries and setting variables


In [74]:
import os
import numpy as np

# Path for exported data, numpy arrays
DATA_PATH = 'BSL_Data'  # Update with the correct path to 'MP_Data' directory

# Creating directories for data organization


In [75]:
actions = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T','U', 'V', 'W', 'X', 'Y', 'Z'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [17]:
# Folder start
start_folder = 30

for action in actions: 
    action_path = os.path.join(DATA_PATH, action)
    
    try:
        
        
        # Find the maximum directory number
        dirmax = np.max(np.array(os.listdir(action_path)).astype(int))
    except (FileNotFoundError, ValueError):
        dirmax = start_folder - 1  # Set dirmax to start_folder - 1 if the action directory doesn't exist or is empty
    
    for sequence in range(1, no_sequences + 1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax + sequence)))
        except FileExistsError:
            pass  # Directory already exists, move on to the next one


SyntaxError: invalid syntax (2274782276.py, line 7)

# Recording Data with MediaPipe and OpenCV


In [23]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Show prompt on webcam feed to press 'g' to start recording
        prompt_image = np.zeros((480, 640, 3), dtype=np.uint8)
        
        cv2.putText(prompt_image, f"Press 'g' to start recording data for '{action}'.", (50, 240),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Data Collection', prompt_image)
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('g'):
                break
            elif key == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                exit()
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [24]:
cap.release()
cv2.destroyAllWindows()

# Data Preparation: Splitting and Categorical Encoding


In [76]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [77]:

label_map = {label:num for num, label in enumerate(actions)}

In [78]:

label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

# Loading Sequences and Labels for Training


In [79]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [80]:
np.array(sequences).shape

(780, 30, 126)

In [81]:
np.array(labels).shape


(780,)

In [82]:
X = np.array(sequences)

In [83]:

X.shape

(780, 30, 126)

In [84]:
y = to_categorical(labels).astype(int)

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [86]:

y_test.shape

(39, 26)

# LSTM Model Construction with Keras


In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [88]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [89]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.summary()

C:\Users\HP\anaconda3\ANaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 30, 64)         │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 204,218 (797.73 KB)

 Trainable params: 204,218 (797.73 KB)

 Non-trainable params: 0 (0.00 B)

In [90]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Fit the model
history = model.fit(X_train, y_train, epochs=250)

Epoch 1/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - categorical_accuracy: 0.0461 - loss: 3.2215
Epoch 2/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - categorical_accuracy: 0.1073 - loss: 2.9956
Epoch 3/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - categorical_accuracy: 0.1270 - loss: 2.9723
Epoch 4/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - categorical_accuracy: 0.1142 - loss: 2.8039
Epoch 5/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - categorical_accuracy: 0.1873 - loss: 2.4913
Epoch 6/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - categorical_accuracy: 0.2609 - loss: 2.3400
Epoch 7/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - categorical_accuracy: 0.2970 - loss: 2.2674
Epoch 8/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 0.2299 - loss: 2.8119
Epoch 9/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - categorical_accuracy: 0.3284 - loss: 2.2283
Epoch 10/250
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - categorical_accuracy: 0.3982 - loss: 1.8504
Epoch 11/250
24/24

In [91]:
actions.shape[0]

26

In [92]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 30, 64)         │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 612,656 (2.34 MB)

 Trainable params: 204,218 (797.73 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 408,438 (1.56 MB)

In [93]:
model.save('action10.h5')

# Making Predictions on Test Data


In [94]:
res = model.predict(X_test)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step


In [95]:
actions[np.argmax(res[4])]

'L'

In [96]:
actions[np.argmax(y_test[4])]

'L'

In [97]:
model.load_weights('action10.h5')

In [98]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [99]:
yhat = model.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [100]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [101]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[36,  0],
        [ 0,  3]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 1,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  1],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 1,  1]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  2]],

       [[36,  0],
        [ 0,  3]],

       [[36,  1],
        [ 0,  2]],

       [[36,  0],
        [ 0,  3]],

       [[37,  0],
        [ 0,  2]]], dtype=int64)

In [102]:

accuracy_score(ytrue, yhat)

0.9487179487179487

# Real-time Sign Language Recognition with MediaPipe and Keras


In [103]:

from scipy import stats

In [104]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



In [59]:
# Initialize variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Open the webcam feed
cap = cv2.VideoCapture(0)

# Set up mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Check if no keypoints are detected
        if not results:
            continue

        # Extract keypoints and update sequence
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        # Perform prediction if sequence length is met
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
            

            # Update sentence if prediction meets threshold and conditions
            if len(predictions) >= 10 and np.unique(predictions[-10:])[0] == np.argmax(res) and res[np.argmax(res)] > threshold:
                if len(sentence) > 0 and actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
                
                # Limit sentence length to 5
                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Visualize probabilities
               # image = prob_viz(res, alphabets, image, colors)

        # Display sentence on the screen
        cv2.rectangle(image, (0, 0), (640, 40), (0, 255, ), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show image with annotations
        cv2.imshow('OpenCV Feed', image)

        # Break loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources and close windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━